In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import os,re
import docx2txt
import textract
import pdfminer
import pandas as pd
import numpy as np
from docx import Document
import sys
from tqdm import tqdm
import string
import seaborn as sns
import spacy

In [2]:
folder_Name = []
resume = []
Folder_Code = []

class_names = ['Peoplesoft resumes', 'React JS resumes', 'SQL Developer Lightning insight', 'workday resumes']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

print (class_names_label)

number_of_classes = len(class_names)

{'Peoplesoft resumes': 0, 'React JS resumes': 1, 'SQL Developer Lightning insight': 2, 'workday resumes': 3}


In [3]:
def load_data():
    
    dataset = "E:\\backup\\OneDrive\\Desktop\\Data Science\\Project\\Resume Classification\\Resumes-20211103T133301Z-001.zip (Unzipped Files) - Copy - Copy\\Resumes"
        
    for folder in os.listdir(dataset):
        label = class_names_label[folder]
        for file in tqdm(os.listdir(os.path.join(dataset, folder))):
            file_path = os.path.join(os.path.join(dataset, folder), file)
            text = docx2txt.process(file_path)
            resume.append(text)
            folder_Name.append(folder)
            Folder_Code.append(label)

In [4]:
load_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 36.24it/s]


In [5]:
data = {'Resumes': resume,
        'Category': folder_Name,
        'Folder Code': Folder_Code}
df = pd.DataFrame(data=data,columns=['Resumes','Category','Folder Code'])

In [6]:
df['cleaned_resume'] = ''

In [7]:
import re
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)
    resumeText = re.sub('RT|cc', ' ', resumeText)
    resumeText = re.sub('#\S+', '', resumeText)
    resumeText = re.sub('@\S+', '  ', resumeText)
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)
    return resumeText
    
df['cleaned_resume'] = df.Resumes.apply(lambda x: cleanResume(x))

In [9]:
df_1 = df.copy()

In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df_1['cleaned_resume'] = df_1['cleaned_resume'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

requiredText = df_1['cleaned_resume'].values
requiredTarget = df_1['Folder Code'].values

In [12]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english')
word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)


X_train,X_test,y_train,y_test = train_test_split(WordFeatures,requiredTarget,random_state=42, test_size=0.3,
                                                 shuffle=True, stratify=requiredTarget)

In [14]:
from sklearn.naive_bayes import MultinomialNB as MB
model = OneVsRestClassifier(MB())
model.fit(X_train, y_train)


OneVsRestClassifier(estimator=MultinomialNB())

In [16]:
import pickle

In [17]:
model1 = 'finalized_model.sav'
pickle.dump(model, open(model1, 'wb'))

In [18]:
vector = 'vectorizer.sav'
pickle.dump(word_vectorizer, open(vector, 'wb'))